# Polygonscan API Data Retrieval

In [1]:
import requests, pandas as pd, time, math, csv, datetime

In [2]:
polygonscan_token = '5PDGJUG4D8ZBVDVXMZEIFV8FUTDDUC2XJE'

# Polygon block range
startblock = 0
endblock = 44990000 # Jul-12-2023 12:55:30 PM +UTC

# Session object
session = requests.Session()

# All addresses
df = pd.read_csv('../data/subsets.csv')
polygon_addresses = df.loc[df['Polygon'] == 1, 'Address'].tolist()
print(f'Number of addresses recorded on Polygon: {len(polygon_addresses)}')

Number of addresses recorded on Polygon: 129988


### ERC-20 Transfers

In [5]:
# Variables
start = 0
batch_size = 10000
end = start + batch_size
iteration = math.ceil(len(polygon_addresses) / batch_size)
errors = []

for j in range(1,iteration):
    print(f'Iteration {j} started')
    data = [["timeStamp", "hash", "from", "to", "contractAddress", "value", "nonce", "tokenName", "tokenDecimal", "gasPrice", "gasUsed", "chainName", "tokenType"]]
    for i in range(start,end):
        try:
            api_url = f'https://api.polygonscan.com/api?module=account&action=tokentx&address={polygon_addresses[i]}&endblock={endblock}&apikey={polygonscan_token}'
            # make the API request and extract the transaction data
            response = session.get(api_url)
            transfers = response.json()['result']
            token_transfer_data = [(tx['timeStamp'], tx['hash'], tx['from'], tx['to'], tx['contractAddress'], tx['value'], tx['nonce'],tx['tokenName'], tx['tokenDecimal'], tx['gasPrice'], tx['gasUsed'], 'Polygon', '20') for tx in transfers]
            data.extend(token_transfer_data)
            time.sleep(0.02)
            if i%1000==0:
                current_time = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
                print(f'{current_time} Addresses done: {i} / {len(polygon_addresses)} ({i/len(polygon_addresses)*100} %)')

        except Exception as e:
            print(f"Error in {polygon_addresses[i]}: {str(e)}")
            errors.append(polygon_addresses[i])
            continue

    # Write csv file
    with open(f'../data/polygon_erc20_{start}-{end}.csv', 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerows(data)
    print('CSV successfully written')

    start = end + 1
    end += batch_size
    if end > len(polygon_addresses) - 1:
        end = len(polygon_addresses) - 1

Iteration 1 started
2023-07-12 22:16:42 Addresses done: 11000 / 129988 (8.46231959873219 %)
2023-07-12 22:20:33 Addresses done: 12000 / 129988 (9.231621380435119 %)
2023-07-12 22:24:45 Addresses done: 13000 / 129988 (10.000923162138044 %)
2023-07-12 22:28:50 Addresses done: 14000 / 129988 (10.77022494384097 %)
2023-07-12 22:32:38 Addresses done: 15000 / 129988 (11.539526725543896 %)
2023-07-12 22:36:41 Addresses done: 16000 / 129988 (12.308828507246822 %)
2023-07-12 22:40:31 Addresses done: 17000 / 129988 (13.07813028894975 %)
2023-07-12 22:44:29 Addresses done: 18000 / 129988 (13.847432070652676 %)
2023-07-12 22:48:26 Addresses done: 19000 / 129988 (14.616733852355601 %)
2023-07-12 22:52:18 Addresses done: 20000 / 129988 (15.386035634058528 %)
CSV successfully written
Iteration 2 started
2023-07-12 22:56:10 Addresses done: 21000 / 129988 (16.155337415761455 %)
2023-07-12 23:00:04 Addresses done: 22000 / 129988 (16.92463919746438 %)
2023-07-12 23:04:26 Addresses done: 23000 / 129988 (1

### ERC-721 Transfers

In [6]:
# Variables
start = 0
batch_size = 20000
end = start + batch_size
iteration = math.ceil(len(polygon_addresses) / batch_size)
print(iteration)
errors = []

for j in range(0,iteration):
    data = [["timeStamp", "hash", "from", "to","contractAddress", "tokenID", "nonce", "tokenName", "gasPrice", "gasUsed", "chainName", "tokenType"]]
    for i in range(start,end):
        try:
            api_url = f'https://api.polygonscan.com/api?module=account&action=tokennfttx&address={polygon_addresses[i]}&endblock={endblock}&apikey={polygonscan_token}'
            response = session.get(api_url)
            transfers = response.json()['result']
            token_transfer_data = [(tx['timeStamp'], tx['hash'], tx['from'], tx['to'], tx['contractAddress'], tx['tokenID'], tx['nonce'], tx['tokenName'], tx['gasPrice'],tx['gasUsed'], 'Polygon', '721') for tx in transfers]
            data.extend(token_transfer_data)
            time.sleep(0.02)
            if i%1000==0:
                current_time = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
                print(f'{current_time} Addresses done: {i} / {len(polygon_addresses)} ({i/len(polygon_addresses)*100} %)')

        except Exception as e:
            print(f"Error in {polygon_addresses[i]}: {str(e)}")
            errors.append(polygon_addresses[i])
            continue

    # Write csv file
    with open(f'../data/polygon_erc721_{start}-{end}.csv', 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerows(data)
    print('CSV successfully written')

    start = end + 1
    end += batch_size
    if end > len(polygon_addresses) - 1:
        end = len(polygon_addresses) - 1

7
2023-07-13 07:47:10 Addresses done: 0 / 129988 (0.0 %)
2023-07-13 07:50:59 Addresses done: 1000 / 129988 (0.7693017817029264 %)
2023-07-13 07:54:49 Addresses done: 2000 / 129988 (1.5386035634058528 %)
2023-07-13 07:58:29 Addresses done: 3000 / 129988 (2.3079053451087796 %)
2023-07-13 08:02:14 Addresses done: 4000 / 129988 (3.0772071268117056 %)
2023-07-13 08:06:07 Addresses done: 5000 / 129988 (3.846508908514632 %)
2023-07-13 08:09:56 Addresses done: 6000 / 129988 (4.615810690217559 %)
2023-07-13 08:13:34 Addresses done: 7000 / 129988 (5.385112471920485 %)
2023-07-13 08:17:24 Addresses done: 8000 / 129988 (6.154414253623411 %)
2023-07-13 08:21:34 Addresses done: 9000 / 129988 (6.923716035326338 %)
2023-07-13 08:25:17 Addresses done: 10000 / 129988 (7.693017817029264 %)
2023-07-13 08:28:47 Addresses done: 11000 / 129988 (8.46231959873219 %)
2023-07-13 08:32:33 Addresses done: 12000 / 129988 (9.231621380435119 %)
2023-07-13 08:36:26 Addresses done: 13000 / 129988 (10.000923162138044 %)

### ERC-1155 transfers

In [7]:
# Variables
start = 0
batch_size = 20000
end = start + batch_size
iteration = math.ceil(len(polygon_addresses) / batch_size)
errors = []

for j in range(0,iteration):
    print(f'Iteration {j} started')
    data = [["timeStamp", "hash", "from", "to", "contractAddress", "tokenID", "tokenValue", "tokenName", "nonce", "gasPrice", "gasUsed", "chainName", "tokenType"]]
    for i in range(start,end):
        try:
            api_url = f'https://api.polygonscan.com/api?module=account&action=token1155tx&address={polygon_addresses[i]}&endblock={endblock}&apikey={polygonscan_token}'
            # make the API request and extract the transaction data
            response = session.get(api_url)
            transfers = response.json()['result']
            token_transfer_data = [(tx['timeStamp'], tx['hash'], tx['from'], tx['to'], tx['contractAddress'], tx['tokenID'], tx['tokenValue'], tx['tokenName'], tx['nonce'], tx['gasPrice'], tx['gasUsed'], "Polygon", "1155" ) for tx in transfers]
            data.extend(token_transfer_data)
            time.sleep(0.02)
            if i%1000==0:
                current_time = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
                print(f'{current_time} Addresses done: {i} / {len(polygon_addresses)} ({i/len(polygon_addresses)*100} %)')

        except Exception as e:
            print(f"Error in {polygon_addresses[i]}: {str(e)}")
            errors.append(polygon_addresses[i])
            continue

    # Write csv file
    with open(f'../data/polygon_erc1155_{start}-{end}.csv', 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerows(data)
    print(f'CSV successfully written {datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")}')

    start = end + 1
    end += batch_size
    if end > len(polygon_addresses) - 1:
        end = len(polygon_addresses) - 1

Iteration 0 started
2023-07-13 15:24:27 Addresses done: 0 / 129988 (0.0 %)
2023-07-13 15:27:28 Addresses done: 1000 / 129988 (0.7693017817029264 %)
2023-07-13 15:30:40 Addresses done: 2000 / 129988 (1.5386035634058528 %)
2023-07-13 15:33:45 Addresses done: 3000 / 129988 (2.3079053451087796 %)
2023-07-13 15:36:50 Addresses done: 4000 / 129988 (3.0772071268117056 %)
2023-07-13 15:40:00 Addresses done: 5000 / 129988 (3.846508908514632 %)
2023-07-13 15:43:01 Addresses done: 6000 / 129988 (4.615810690217559 %)
2023-07-13 15:46:10 Addresses done: 7000 / 129988 (5.385112471920485 %)
2023-07-13 15:49:17 Addresses done: 8000 / 129988 (6.154414253623411 %)
2023-07-13 15:52:28 Addresses done: 9000 / 129988 (6.923716035326338 %)
2023-07-13 15:55:28 Addresses done: 10000 / 129988 (7.693017817029264 %)
2023-07-13 15:58:22 Addresses done: 11000 / 129988 (8.46231959873219 %)
2023-07-13 16:01:20 Addresses done: 12000 / 129988 (9.231621380435119 %)
2023-07-13 16:04:33 Addresses done: 13000 / 129988 (10.

### Transactions

In [8]:
# Variables
start = 0
batch_size = 20000
end = start + batch_size
iteration = math.ceil(len(polygon_addresses) / batch_size)
print(iteration)
errors = []

for j in range(0,iteration):
    print(f'Iteration {j} started')
    data = [["hash", "from", "to", "timeStamp", "nonce", "value", "gasPrice", "input", "gasUsed", "functionName", "chainName"]]
    for i in range(start,end):
        try:
            api_url = f'https://api.polygonscan.com/api?module=account&action=txlist&address={polygon_addresses[i]}&endblock={endblock}&apikey={polygonscan_token}'
            # make the API request and extract the transaction data
            response = session.get(api_url)
            transactions = response.json()['result']
            tx_data = [(tx['hash'], tx['from'], tx['to'], tx['timeStamp'], tx['nonce'], tx['value'], tx['gasPrice'], tx['input'], tx['gasUsed'], tx['functionName'], "Polygon") for tx in transactions]
            data.extend(tx_data)
            time.sleep(0.02)
            if i%1000==0:
                current_time = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
                print(f'{current_time} Addresses done: {i} / {len(polygon_addresses)} ({i/len(polygon_addresses)*100} %)')
        except Exception as e:
            print(f"Error in {polygon_addresses[i]}: {str(e)}")
            errors.append(polygon_addresses[i])
            continue

    # Write csv file
    with open(f'../data/transactions_polygon_{start}-{end}.csv', 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerows(data)
    print('CSV successfully written')

    start = end + 1
    end += batch_size
    if end > len(polygon_addresses) - 1:
        end = len(polygon_addresses) - 1

7
Iteration 0 started
2023-07-13 22:48:56 Addresses done: 0 / 129988 (0.0 %)
2023-07-13 22:51:54 Addresses done: 1000 / 129988 (0.7693017817029264 %)
2023-07-13 22:55:14 Addresses done: 2000 / 129988 (1.5386035634058528 %)
2023-07-13 22:58:18 Addresses done: 3000 / 129988 (2.3079053451087796 %)
2023-07-13 23:01:33 Addresses done: 4000 / 129988 (3.0772071268117056 %)
2023-07-13 23:04:54 Addresses done: 5000 / 129988 (3.846508908514632 %)
2023-07-13 23:08:04 Addresses done: 6000 / 129988 (4.615810690217559 %)
2023-07-13 23:11:29 Addresses done: 7000 / 129988 (5.385112471920485 %)
2023-07-13 23:15:04 Addresses done: 8000 / 129988 (6.154414253623411 %)
2023-07-13 23:18:26 Addresses done: 9000 / 129988 (6.923716035326338 %)
2023-07-13 23:21:36 Addresses done: 10000 / 129988 (7.693017817029264 %)
2023-07-13 23:24:27 Addresses done: 11000 / 129988 (8.46231959873219 %)
2023-07-13 23:27:39 Addresses done: 12000 / 129988 (9.231621380435119 %)
2023-07-13 23:30:52 Addresses done: 13000 / 129988 (1